# TP2 - DB Normalization and Querying

The objectives of this TP are:
1. Apply normalization 1NF -> 2NF -> 3NF
2. Perform SQL queries on the normalized database

In this TP, we will use a database **`wine.db`** (available in the course's website) containing wine information related to 'production' and 'sales'. 

> Production <---> Wine <---> Sales


---

### Working with db files in Jupyter
- Python provides an interface for SQLite through the *sqlite3* module
- The **`%%sql`** magic builds upon it (and other tools) to enable the usage of SQL commands within a Jupyter Notebook as in common SQL clients.
- Before proceeding, make sure that **`wine.db`** is in the same path as this notebook.
  - If **`wine.db`** is not in the same path, an empty **`wine.db`** file will be created, resulting in errors in later steps of the TP.
- The SQLite module in Python commits transactions automatically, this means that any change in the DB is immediately written to the file, e.g. creating/deleting tables.
  -  For this reason, it is recommended to have a backup of **`wine.db`** as it is provided in the course's website.

---

**`wine.db`** contains the following unnormalized tables:

<center>**Master1**</center>

|*Attribute*|         *Description*          |
| -------   |--------------------------------|
| NV        | Wine number                    |
| CRU       | Vineyard or group of vineyards |
| DEGRE     | Alcohol content                |
| MILL      | Vintage year                   |
| QTE       | Number of bottles harvested    |
| NP        | Producer number                |
| NOM       | Producer's last name           |
| PRENOM    | Producer's first name          |
| REGION    | Production region              |

From wikipedia:

__Cru__: Often used to indicate a specifically named and legally defined vineyard or ensemble of vineyards and the vines "which grow on [such] a reputed terroir; by extension of good quality." The term is also used to refer to the wine produced from such vines.


<center>**Master2**</center>

|*Attribute*|                         *Description*                  |
| -------   |--------------------------------------------------------|
| NV        | Wine number                                            |
| CRU       | Vineyard or group of vineyards                         |
| DEGRE     | Alcohol content                                        |
| MILL      | Vintage year                                           |
| DATES     | Buying date                                            |
| LIEU      | Place where the wine was sold                          |
| QTE       | Number of bottles bought                               |
| NB        | Client (buveur) number                                 |
| NOM       | Client's last name                                     |
| PRENOM    | Client's first name                                    |
| TYPE      | Type of client by volume of purchases                  |
| REGION    | Administrative Region (different to production region) |


In [1]:
import sqlite3    # Python interface for SQLite databases

In [2]:
def printSchema(connection):
    # Function to print the DB schema
    # Source: http://stackoverflow.com/a/35092773/4765776
    for (tableName,) in connection.execute(
        """
        select NAME from SQLITE_MASTER where TYPE='table' order by NAME;
        """
    ):
        print("{}:".format(tableName))
        for (
            columnID, columnName, columnType,
            columnNotNull, columnDefault, columnPK,
        ) in connection.execute("pragma table_info('{}');".format(tableName)):
            print("  {id}: {name}({type}){null}{default}{pk}".format(
                id=columnID,
                name=columnName,
                type=columnType,
                null=" not null" if columnNotNull else "",
                default=" [{}]".format(columnDefault) if columnDefault else "",
                pk=" *{}".format(columnPK) if columnPK else "",
            ))

In [3]:
conn = sqlite3.connect('wine.db')
c = conn.cursor()
print("Database schema:")
printSchema(conn)           # An usefull way to viualize the content of the database

Database schema:
MASTER1:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: QTE(NUM)
  5: NP(NUM)
  6: NOM(TEXT)
  7: PRENOM(TEXT)
  8: REGION(TEXT)
MASTER2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: DATES(DATE)
  5: LIEU(TEXT)
  6: QTE(NUM)
  7: NB(NUM)
  8: NOM(TEXT)
  9: PRENOM(TEXT)
  10: TYPE(TEXT)
  11: REGION(TEXT)


From this point we will use __%%sql__ magic

In [4]:
%load_ext sql
%sql sqlite:///wine.db

'Connected: @wine.db'

# PART I: Database normalization

The first task on this TP is the normalization of the wine data. In its current state both tables **Master1** and **Master2** are in the First Normal Form (1NF).

By inspecting the content of these tables we can see that multiple tuples have NULL values.

In [5]:
%%sql SELECT *
FROM Master1
LIMIT 10;

 * sqlite:///wine.db
Done.


NV,CRU,DEGRE,MILL,QTE,NP,NOM,PRENOM,REGION
None,None,None,None,None,3,Six,Paul,Alsace
None,None,None,None,None,6,Marmagne,Bernard,Bourgogne
None,None,None,None,None,8,Lioger d'Harduy,Gabriel,Bourgogne
None,None,None,None,None,16,Barbin,Bernard,Bourgogne
None,None,None,None,None,17,Faiveley,Guy,Bourgogne
None,None,None,None,None,18,Tramier,Jean,Bourgogne
None,None,None,None,None,19,Dupaquier,Roger,Bourgogne
None,None,None,None,None,20,Lamy,Jean,Bourgogne
None,None,None,None,None,21,Cornu,Edmond,Bourgogne
None,None,None,None,None,26,Violot,Gilbert,Bourgogne


* Notice that Jupyter *displays* 'None' instead of 'NULL'. 
  - This is only to comply with python notation.
* To account for NULL values, your SQL queries must test explicitly for 'NULL'.

Another problem in **Master1** and **Master2** is data redundancy, for example:

In [6]:
%%sql SELECT *
FROM Master1
WHERE NV = 45;

 * sqlite:///wine.db
Done.


NV,CRU,DEGRE,MILL,QTE,NP,NOM,PRENOM,REGION
45,Chiroubles,None,1983,90,2,Boxler,Albert,Alsace
45,Chiroubles,None,1983,912,67,Descombes,Jean Ernest,Beaujolais
45,Chiroubles,None,1983,98,71,Chalandard,Danile,Jura
45,Chiroubles,None,1983,540,78,Michlel,Pierre Emile,Jura
45,Chiroubles,None,1983,450,86,Dumazet,Marc,Rhone


In [7]:
%%sql SELECT *
FROM Master1
;

 * sqlite:///wine.db
Done.


NV,CRU,DEGRE,MILL,QTE,NP,NOM,PRENOM,REGION
None,None,None,None,None,3,Six,Paul,Alsace
None,None,None,None,None,6,Marmagne,Bernard,Bourgogne
None,None,None,None,None,8,Lioger d'Harduy,Gabriel,Bourgogne
None,None,None,None,None,16,Barbin,Bernard,Bourgogne
None,None,None,None,None,17,Faiveley,Guy,Bourgogne
None,None,None,None,None,18,Tramier,Jean,Bourgogne
None,None,None,None,None,19,Dupaquier,Roger,Bourgogne
None,None,None,None,None,20,Lamy,Jean,Bourgogne
None,None,None,None,None,21,Cornu,Edmond,Bourgogne
None,None,None,None,None,26,Violot,Gilbert,Bourgogne


---

Additional resource for Normalization:

https://www.youtube.com/watch?v=UrYLYV7WSHM

---

#### Exercise 1.1

Convert table **Master1** to the Second Normal Form (2NF) or Third Normal Form (3NF) as applicable.
* Explain your answer
* List main functional dependencies (not all of them)
* Describe the schema of new tables and how they relate
  * You can write Tables as above or you can insert images in the notebook.
  
Remember that **`wine.db`** contains information related to wine 'production' and 'sells'.

> Production <---> Wine <---> Sales

A good start point is to look for the 'Wine' attributes.

**Hint:** Look for redundant information between the master tables.

#### Exercise 1.2

Convert table **Master2** to the Second Normal Form (2NF) or Third Normal Form (3NF) as applicable.
* Explain your answer
* List main functional dependencies (not all of them)
* Describe the schema of new tables and how they relate
  * You can write Tables as above or you can insert images in the notebook.

**Note:** For this part, consider that a wine can be bought in multiple locations and multiple times per day.

Once you have defined the 2NF or 3NF (as applicable) we need to split the data into new tables.

A table can be created from the result of a query.

In the following example we will create a new table "dummy" to store the different values of alcohol content.

In [8]:
%%sql 
DROP TABLE IF EXISTS dummy;

-- Create dummy table
CREATE TABLE dummy AS
SELECT DISTINCT DEGRE
FROM MASTER1;

 * sqlite:///wine.db
Done.
Done.


[]

In [9]:
%%sql 
SELECT * FROM transac;

 * sqlite:///wine.db
(sqlite3.OperationalError) no such table: transac
[SQL: SELECT * FROM transac;]
(Background on this error at: http://sqlalche.me/e/e3q8)


In [10]:
print("\nContent of the database")
printSchema(conn)


Content of the database
MASTER1:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: QTE(NUM)
  5: NP(NUM)
  6: NOM(TEXT)
  7: PRENOM(TEXT)
  8: REGION(TEXT)
MASTER2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: DATES(DATE)
  5: LIEU(TEXT)
  6: QTE(NUM)
  7: NB(NUM)
  8: NOM(TEXT)
  9: PRENOM(TEXT)
  10: TYPE(TEXT)
  11: REGION(TEXT)
dummy:
  0: DEGRE(NUM)


In [11]:
%%sql 
DROP TABLE IF EXISTS transactio;

 * sqlite:///wine.db
Done.


[]

In [12]:
%%sql
SELECT *
FROM dummy;

 * sqlite:///wine.db
Done.


DEGRE
None
11.5
11.3
12.1
10.9
11.7
11.2
12.3
11.9
11.8


Looking into "dummy", we notice that our query includes NULL. This is not allowed if we were to use DEGRE as key for a table.

To correct this, we need to change the query to explicitly test for NULL as follows:

In [13]:
%%sql DROP TABLE IF EXISTS dummy;

-- Create dummy table
CREATE TABLE dummy AS
SELECT DISTINCT DEGRE
FROM MASTER1
WHERE DEGRE IS NOT NULL;

SELECT *
FROM dummy;

 * sqlite:///wine.db
Done.
Done.
Done.


DEGRE
11.5
11.3
12.1
10.9
11.7
11.2
12.3
11.9
11.8
10.7


Notice that we use `NULL` given that `None` is only used for display.

In [14]:
# Remove "dummy" table
%sql DROP TABLE IF EXISTS dummy;

 * sqlite:///wine.db
Done.


[]

#### Exercise 1.3

Create the new tables from Master1:

In [15]:
%%sql
DROP TABLE IF EXISTS wine;
DROP TABLE IF EXISTS producer;
DROP TABLE IF EXISTS harvest;

CREATE TABLE wine AS
SELECT DISTINCT NV,CRU,DEGRE,MILL
FROM MASTER1
WHERE NV IS NOT NULL;

CREATE TABLE producer AS
SELECT DISTINCT NP,NOM,PRENOM,REGION
FROM MASTER1
WHERE NP IS NOT NULL;


CREATE TABLE harvest AS
SELECT DISTINCT NP,NV,QTE
FROM MASTER1
WHERE NV and NP IS NOT NULL;


 * sqlite:///wine.db
Done.
Done.
Done.
Done.
Done.
Done.


[]

In [17]:
%%sql
DROP VIEW IF EXISTS view_name;
SELECT *
FROM wine
limit 10;
SELECT *
FROM producer
limit 10;
SELECT *
FROM harvest
limit 10;

 * sqlite:///wine.db
Done.
Done.
Done.
Done.


NP,NV,QTE
1,1,300
73,1,1
5,2,100
1,3,400
10,4,35
30,5,46
42,6,300
98,7,60
90,8,12
98,10,100


#### Exercise 1.4

Create the new tables from Master2:

In [18]:
%%sql

DROP TABLE IF EXISTS client;
DROP TABLE IF EXISTS transac;
DROP TABLE IF EXISTS DonneesAdministrative;

CREATE TABLE client AS
SELECT DISTINCT NB,NOM,PRENOM,TYPE
FROM MASTER2
WHERE NB IS NOT NULL;

CREATE TABLE transac AS
SELECT DISTINCT NV,NB,QTE,LIEU,DATES
FROM MASTER2
WHERE NV and NB IS NOT NULL;

CREATE TABLE DonneesAdministrative AS 
SELECT DISTINCT LIEU,REGION
FROM MASTER2;


 * sqlite:///wine.db
Done.
Done.
Done.
Done.
Done.
Done.


[]

# PART II: SQL QUERIES

In the second part of this TP you will create SQL queries to retrieve information from the database.

**Important:**

- You MUST use the normalized tables created in previous steps.
  - The normalized tables will also be used in TP3.
- Do NOT use **Master1** and **Master2** in your queries.

#### Exercise 2.1

What are the different types of clients (buveurs) by volume of purchases?

In [ ]:
%%sql 
SELECT DISTINCT TYPE as Type_of_buveurs
FROM client;

#### Exercise 2.2

What regions produce Pommard or Brouilly?

In [ ]:
%%sql 
SELECT CRU,REGION 
FROM wine 
    JOIN harvest ON wine.NV=harvest.NV
    JOIN producer on harvest.NP=producer.NP
WHERE CRU in ("Pommard","Brouilly"); 

#### Exercise 2.3

What regions produce Pommard and Brouilly?

In [20]:
%%sql 
SELECT REGION
FROM wine 
    JOIN harvest ON wine.NV=harvest.NV
    JOIN producer ON harvest.NP=producer.NP
WHERE CRU in ("Pommard","Brouilly")
GROUP BY REGION
HAVING count(distinct(cru))=2;

 * sqlite:///wine.db
Done.


REGION
Bourgogne


#### Exercise 2.4

Get the number of wines bught by CRU and Millésime

In [ ]:
%%sql 
SELECT CRU,MILL, count(distinct(NB))
FROM wine
    JOIN transac ON wine.NV=transac.NV
GROUP BY CRU,MILL
ORDER BY CRU, MILL;

#### Exercise 2.5

Retrieve the wine number (NV) of wines produced by more than three producers

In [ ]:
%%sql 
SELECT wine.NV, count(distinct(producer.NP)) as Number_of_producers
FROM wine
    JOIN harvest ON wine.NV=harvest.NV
    JOIN producer ON harvest.NP=producer.NP
GROUP BY wine.NV
HAVING Number_of_producers >3;

#### Exercise 2.6

Which producers have not produced any wine?

In [19]:
%%sql 

SELECT NP,NOM,PRENOM
FROM producer 
WHERE NP not in (SELECT producer.NP
FROM producer
    JOIN harvest ON producer.NP=harvest.NP 
WHERE QTE >0);

 * sqlite:///wine.db
Done.


NP,NOM,PRENOM
3,Six,Paul
6,Marmagne,Bernard
8,Lioger d'Harduy,Gabriel
16,Barbin,Bernard
17,Faiveley,Guy
18,Tramier,Jean
19,Dupaquier,Roger
20,Lamy,Jean
21,Cornu,Edmond
26,Violot,Gilbert


#### Exercise 2.7

What clients (buveurs) have bought at least one wine from 1980?

In [ ]:
%%sql 

SELECT DISTINCT client.NB,NOM,PRENOM
FROM client 
    JOIN transac ON client.NB=transac.NB
    JOIN wine ON transac.NV=wine.NV
WHERE MILL=1980
limit 10;

#### Exercise 2.8

What clients (buveurs) have NOT bought any wine from 1980?

In [ ]:
%%sql 
SELECT DISTINCT NB,NOM,PRENOM
FROM client
WHERE NB not in(SELECT DISTINCT client.NB
FROM client 
    JOIN transac ON client.NB=transac.NB
    JOIN wine ON transac.NV=wine.NV
WHERE MILL=1980)
ORDER BY NB
;

#### Exercise 2.9

What clients (buveurs) have bought ONLY wines from 1980?

In [ ]:
%%sql 
SELECT DISTINCT client.NB,NOM,PRENOM
FROM client
    JOIN transac ON client.NB=transac.NB
    JOIN wine ON transac.NV=wine.NV
GROUP BY client.NB
having sum(distinct MILL)%1980=0;

#### Exercise 2.10

List all wines from 1980

In [ ]:
%%sql 
SELECT NV,CRU,MILL,DEGRE
FROM wine
WHERE MILL=1980;

#### Exercise 2.11

What are the wines from 1980 bought by NB=2?

In [ ]:
%%sql 
SELECT wine.NV,CRU,MILL,DEGRE
FROM wine
    JOIN transac ON wine.NV=transac.NV
WHERE MILL=1980 and NB = 2;

#### Exercise 2.12

What clients (buveurs) have bought ALL the wines from 1980?

In [ ]:
%%sql 
SELECT client.NB,client.NOM,client.PRENOM
FROM wine
    JOIN transac ON wine.NV=transac.NV
    JOIN client ON transac.NB = client.NB
GROUP BY client.nom,client.PRENOM
HAVING wine.MILL=1980 and count(wine.mill) = (SELECT COUNT(*) FROM wine WHERE mill = 1980)